In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Gene_dict_occur.npy
Gene_dict_occur.npz
GV_in_text.csv
GV_in_text.npz
kaggle_train.csv
properties_2016.csv
properties_2017.csv
quora_sample_submission.csv
quora_train.csv
sample_submission.csv
stage1_solution_filtered.csv
stage2_sample_submission.csv
stage2_test_text.csv
stage2_test_variants.csv
submissionFile
test.p
test_variants
text_test_arrays.npy
text_train_arrays.npy
Thesaurus.txt
train_2016_v2.csv
train_2017.csv
training_text
training_variants
train.p
zillow_data_dictionary.xlsx
zillow_y_final.csv



In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from catboost import CatBoostRegressor
from tqdm import tqdm
import gc
import datetime as dt

print('Loading Properties ...')
properties2016 = pd.read_csv('../input/properties_2016.csv', low_memory = False)
properties2017 = pd.read_csv('../input/properties_2017.csv', low_memory = False)

print('Loading Train ...')
train2016 = pd.read_csv('../input/train_2016_v2.csv', parse_dates=['transactiondate'], low_memory=False)
train2017 = pd.read_csv('../input/train_2017.csv', parse_dates=['transactiondate'], low_memory=False)

def add_date_features(df):
    df["transaction_year"] = df["transactiondate"].dt.year
    df["transaction_month"] = (df["transactiondate"].dt.year - 2016)*12 + df["transactiondate"].dt.month
    #df["transaction_day"] = df["transactiondate"].dt.day
    df["transaction_quarter"] = (df["transactiondate"].dt.year - 2016)*4 +df["transactiondate"].dt.quarter
    df.drop(["transactiondate"], inplace=True, axis=1)
    return df

train2016 = add_date_features(train2016)
train2017 = add_date_features(train2017)

print('Loading Sample ...')
sample_submission = pd.read_csv('../input/sample_submission.csv', low_memory = False)

print('Merge Train with Properties ...')
train2016 = pd.merge(train2016, properties2016, how = 'left', on = 'parcelid')
train2017 = pd.merge(train2017, properties2017, how = 'left', on = 'parcelid')

print('Tax Features 2017  ...')
#train2017.iloc[:, train2017.columns.str.startswith('tax')] = np.nan

print('Concat Train 2016 & 2017 ...')
train_df = pd.concat([train2016, train2017], axis = 0)
test_df = pd.merge(sample_submission[['ParcelId']], properties2016.rename(columns = {'parcelid': 'ParcelId'}), how = 'left', on = 'ParcelId')

del properties2016, properties2017, train2016, train2017
gc.collect();

Loading Properties ...
Loading Train ...
Loading Sample ...
Merge Train with Properties ...
Tax Features 2017  ...
Concat Train 2016 & 2017 ...


In [3]:
#Additional Feature
def add_feature(input_):
    output_ = pd.DataFrame()
    #life of property
    #output_['N-life'] = 2018 - input_['yearbuilt']

    #error in calculation of the finished living area of home
    output_['N-LivingAreaError'] = input_['calculatedfinishedsquarefeet']/input_['finishedsquarefeet12']

    #proportion of living area
    output_['N-LivingAreaProp'] = input_['calculatedfinishedsquarefeet']/input_['lotsizesquarefeet']
    output_['N-LivingAreaProp2'] = input_['finishedsquarefeet12']/input_['finishedsquarefeet15']

    #Amout of extra space
    output_['N-ExtraSpace'] = input_['lotsizesquarefeet'] - input_['calculatedfinishedsquarefeet'] 
    output_['N-ExtraSpace-2'] = input_['finishedsquarefeet15'] - input_['finishedsquarefeet12'] 

    #Total number of rooms
    output_['N-TotalRooms'] = input_['bathroomcnt']+input_['bedroomcnt']

    #Average room size
    output_['N-AvRoomSize'] = input_['calculatedfinishedsquarefeet']/output_['N-TotalRooms']

    # Number of Extra rooms
    output_['N-ExtraRooms'] =  output_['N-TotalRooms'] -input_['roomcnt']

    #Ratio of the built structure value to land area
    output_['N-ValueProp'] = input_['structuretaxvaluedollarcnt']/input_['landtaxvaluedollarcnt']

    #Does property have a garage, pool or hot tub and AC?
    output_['N-GarPoolAC'] = ((input_['garagecarcnt']>0) & (input_['pooltypeid10']>0) & (input_['airconditioningtypeid']!=5))*1 

    output_["N-location"] = input_["latitude"] + input_["longitude"]
    output_["N-location-2"] = input_["latitude"]*input_["longitude"]
    #output_["N-location-2round"] = output_["N-location-2"].round(-4)

    #output_["N-latitude-round"] = input_["latitude"].round(-4)
    #output_["N-longitude-round"] = input_["longitude"].round(-4)

    #Lets create additional features based off the tax related variables

    #Ratio of tax of property over parcel
    output_['N-ValueRatio'] = input_['taxvaluedollarcnt']/input_['taxamount']

    #TotalTaxScore
    output_['N-TaxScore'] = input_['taxvaluedollarcnt']*input_['taxamount']


    #Length of time since unpaid taxes
    #output_['N-life'] = 2018 - input_['taxdelinquencyyear']


    #polnomials of tax delinquency year
    #output_["N-taxdelinquencyyear-2"] = output_['N-life'] ** 2
    #output_["N-taxdelinquencyyear-3"] =output_['N-life'] ** 3

    #Other features based off the location
    #Number of properties in the zip
    #zip_count = input_['regionidzip'].value_counts().to_dict()
    #output_['N-zip_count'] = input_['regionidzip'].map(zip_count)

    #Number of properties in the city
    #city_count = input_['regionidcity'].value_counts().to_dict()
    #output_['N-city_count'] = input_['regionidcity'].map(city_count)

    #Number of properties in the county
    #region_count = input_['regionidcounty'].value_counts().to_dict()
    #output_['N-county_count'] = input_['regionidcounty'].map(region_count)

    # More features
    # Mean square feet of neighborhood properties
    #output_['mean_area'] = input_['regionidneighborhood'].map(meanarea)
    # Median year of construction of neighborhood properties
    #output_['med_year'] = input_['regionidneighborhood'].map(medyear)
    # Neighborhood latitude and longitude
    #output_['med_lat'] = input_['regionidneighborhood'].map(medlat)
    #output_['med_long'] = input_['regionidneighborhood'].map(medlong)



    #Indicator whether it has AC or not
    #output_['N-ACInd'] = (input_['airconditioningtypeid']!=5)*1

    #Indicator whether it has Heating or not 
    #output_['N-HeatInd'] = (input_['heatingorsystemtypeid']!=13)*1

    #There's 25 different property uses - let's compress them down to 4 categories
    output_['N-PropType'] = input_.propertylandusetypeid.replace({31 : "Mixed", 46 : "Other", 47 : "Mixed", 
                                                                     246 : "Mixed", 247 : "Mixed", 248 : "Mixed",
                                                                     260 : "Home", 261 : "Home", 262 : "Home", 
                                                                     263 : "Home", 264 : "Home", 265 : "Home",
                                                                     266 : "Home", 267 : "Home", 268 : "Home",
                                                                     269 : "Not Built", 270 : "Home", 271 : "Home",
                                                                     273 : "Home", 274 : "Other", 275 : "Home", 
                                                                     276 : "Home", 279 : "Home", 290 : "Not Built", 
                                                                     291 : "Not Built" })
    #output_ = pd.concat([output_,pd.get_dummies(output_['N-PropType'])],1).drop(['N-PropType'],1)

    #polnomials of the variable
    #output_["N-structuretaxvaluedollarcnt-2"] = input_["structuretaxvaluedollarcnt"] ** 2
    #output_["N-structuretaxvaluedollarcnt-3"] = input_["structuretaxvaluedollarcnt"] ** 3
    #output_["N-structuretaxvaluedollarcnt-log"] = np.log(input_["structuretaxvaluedollarcnt"])
    #Average structuretaxvaluedollarcnt by city
    #group = input_.groupby('regionidcity')['structuretaxvaluedollarcnt'].aggregate('mean').to_dict()
    #output_['N-Avg-structuretaxvaluedollarcnt'] = input_['regionidcity'].map(group)


    #Deviation away from average
    #output_['N-Dev-structuretaxvaluedollarcnt'] = abs((input_['structuretaxvaluedollarcnt'] - output_['N-Avg-structuretaxvaluedollarcnt']))/output_['N-Avg-structuretaxvaluedollarcnt']
    return output_

In [4]:
features1 = add_feature(train_df)
train_df = pd.concat([train_df,features1],1)
train_df.shape

(167888, 77)

In [5]:
print('Remove missing data fields ...')

missing_perc_thresh = 0.98
exclude_missing = []
num_rows = train_df.shape[0]
for c in train_df.columns:
    num_missing = train_df[c].isnull().sum()
    if num_missing == 0:
        continue
    missing_frac = num_missing / float(num_rows)
    if missing_frac > missing_perc_thresh:
        exclude_missing.append(c)
print("We exclude: %s" % len(exclude_missing))

del num_rows, missing_perc_thresh
gc.collect();

Remove missing data fields ...
We exclude: 15


In [6]:
print ("Remove features with one unique value !!")
exclude_unique = []
for c in train_df.columns:
    num_uniques = len(train_df[c].unique())
    if train_df[c].isnull().sum() != 0:
        num_uniques -= 1
    if num_uniques == 1:
        exclude_unique.append(c)
print("We exclude: %s" % len(exclude_unique))

Remove features with one unique value !!
We exclude: 10


In [7]:
print ("Define training features !!")
exclude_other = ['parcelid', 'logerror','propertyzoningdesc']
train_features = []
for c in train_df.columns:
    if c not in exclude_missing \
       and c not in exclude_other and c not in exclude_unique:
        train_features.append(c)
print("We use these for training: %s" % len(train_features))

Define training features !!
We use these for training: 54


In [8]:
print ("Define categorial features !!")
cat_feature_inds = []
cat_unique_thresh = 1000
train_features.remove('N-TotalRooms') 
train_features.remove('N-ExtraRooms') 
train_features.remove('N-GarPoolAC') 
for i, c in enumerate(train_features):
    num_uniques = len(train_df[c].unique())
    if num_uniques < cat_unique_thresh \
       and not 'sqft' in c \
       and not 'cnt' in c \
       and not 'nbr' in c \
       and not 'number' in c:
        cat_feature_inds.append(i)

'', '', ''     
print("Cat features are: %s" % [train_features[ind] for ind in cat_feature_inds])

Define categorial features !!
Cat features are: ['transaction_year', 'transaction_month', 'transaction_quarter', 'airconditioningtypeid', 'buildingqualitytypeid', 'fips', 'heatingorsystemtypeid', 'propertycountylandusecode', 'propertylandusetypeid', 'regionidcity', 'regionidcounty', 'regionidneighborhood', 'regionidzip', 'yearbuilt', 'assessmentyear', 'taxdelinquencyyear', 'N-PropType']


In [9]:
print ("Replacing NaN values by -999 !!")
train_df.fillna(-999, inplace=True)
test_df.fillna(-999, inplace=True)

Replacing NaN values by -999 !!


In [10]:
from sklearn.metrics import mean_absolute_error
print ("Training time !!")
X_train = train_df[train_features]
y_train = train_df.logerror
print(X_train.shape, y_train.shape)

test_df['transactiondate'] = pd.Timestamp('2016-12-01') 
test_df = add_date_features(test_df)
features1 = add_feature(test_df)
test_df = pd.concat([test_df,features1],1)
X_test = test_df[train_features]
print(X_test.shape)

Training time !!
((167888, 51), (167888,))
(2985217, 51)


In [11]:
submission = pd.DataFrame({
    'ParcelId': test_df['ParcelId'],
})
test_dates = {
    '201610': pd.Timestamp('2016-09-30'),
    '201611': pd.Timestamp('2016-10-31'),
    '201612': pd.Timestamp('2016-11-30'),
    '201710': pd.Timestamp('2017-09-30'),
    '201711': pd.Timestamp('2017-10-31'),
    '201712': pd.Timestamp('2017-11-30')
}

    

In [ ]:
models = []
num_ensembles = 20
y_pred = 0.0
for i in tqdm(range(num_ensembles)):
    i = i + 5
    model = CatBoostRegressor(
        iterations=2000, learning_rate=0.01,
        depth=6, l2_leaf_reg=3,
        loss_function='MAE',
        eval_metric='MAE',
        thread_count=24,
        random_seed=i)
    model.fit(
        X_train, y_train,
        cat_features=cat_feature_inds)
    print(mean_absolute_error(model.predict(X_train),y_train))
    models.append(model)

  5%|▌         | 1/20 [05:02<1:35:42, 302.23s/it]

0.0662118834048


 10%|█         | 2/20 [10:05<1:30:46, 302.61s/it]

0.0661868840605


 15%|█▌        | 3/20 [15:07<1:25:40, 302.41s/it]

...

0.0660733065991


 25%|██▌       | 5/20 [25:08<1:15:21, 301.44s/it]

0.066112166828


 30%|███       | 6/20 [30:10<1:10:23, 301.71s/it]

0.066046387818


In [ ]:
y_pred = 0
test_df['transactiondate'] = pd.Timestamp('2016-12-01') 
test_df = add_date_features(test_df)
X_test = test_df[train_features]
for model in models:
    y_pred += model.predict(X_test)
y_pred /= num_ensembles

for label, test_date in test_dates.items():   
    print("Predicting for: %s ... " % (label))    
    submission[label] = y_pred

In [17]:
submission.mean()

ParcelId    1.332586e+07
201612      1.267057e-02
201610      1.267057e-02
201611      1.267057e-02
201712      1.267057e-02
201711      1.267057e-02
201710      1.267057e-02
dtype: float64

In [13]:
# Predict different values for each month.
"""for label, test_date in test_dates.items():
    y_pred = 0
    print("Predicting for: %s ... " % (label))
    
    test_df['transactiondate'] = test_date
    test_df = add_date_features(test_df)
    X_test = test_df[train_features]
    for model in models:
        
        y_pred += model.predict(X_test)
    y_pred /= num_ensembles
    #_preds[test_date] = y_pred
    submission[label] = y_pred"""
    

Predicting for: 201612 ... 
Predicting for: 201610 ... 


KeyboardInterrupt: 

In [ ]:
"""num_ensembles = 1
#y_pred2 = 0.0
for i in tqdm(range(6,10)):
    model = CatBoostRegressor(
        iterations=2000, learning_rate=0.01,
        depth=6, l2_leaf_reg=3,
        loss_function='MAE',
        eval_metric='MAE',
        random_seed=i)
    model.fit(
        X_train, y_train,
        cat_features=cat_feature_inds)
    print(mean_absolute_error(model.predict(X_train),y_train))
    y_pred2 += model.predict(X_test)"""


In [ ]:
"""y_pred2 = y_pred2/5
y_pred3 = (y_pred + y_pred2)/2"""

In [16]:
def get_xgb_imp1(model,feature_names,keep_all = True):
    a = pd.DataFrame(model.feature_importances_)
    
    a.columns = ['feature']
    #a.feature = a.feature.map(feature_rename)
    #a.feature = a.feature.map(lambda x:feature_names[int(x)])
    #a  = a.set_index(a.feature,drop = True)
    #a = a.drop('feature',1)
    b = pd.Series(feature_names)
    #b.index = feature_names
    #if keep_all ==True:
    #    c = a.importance+b
    #else:
    #    c = a.importance
    c = pd.concat([b,a],1)
    c.columns = ['feature','importance']
    c = c.set_index(c['feature'],drop=True)
    c = c.sort_values(ascending=False,by = 'importance')
    return c

In [17]:
get_xgb_imp1(models[0],X_train.columns.values)

,feature,importance
feature,,
regionidzip,regionidzip,6.086402
finishedsquarefeet12,finishedsquarefeet12,5.749457
N-ValueRatio,N-ValueRatio,5.365399
transaction_month,transaction_month,5.324121
taxamount,taxamount,4.610084
yearbuilt,yearbuilt,4.145900
propertycountylandusecode,propertycountylandusecode,3.859135
calculatedfinishedsquarefeet,calculatedfinishedsquarefeet,3.339620
N-AvRoomSize,N-AvRoomSize,3.176372


In [ ]:
get_xgb_imp1(models[0],X_train.columns.values)

In [20]:

submission.to_csv('Only_CatBoost8_2.csv', float_format='%.6f',index=False)

In [23]:
a = pd.read_csv('Only_CatBoost8_2.csv')
b = pd.read_csv('Only_CatBoost7.csv')
c = pd.read_csv('Only_CatBoost6_20.csv')
d = pd.read_csv('Only_CatBoost5.csv')

In [24]:
e = 0.4*a+0.3*b+0.2*c+0.1*a
e.head()

,ParcelId,201612,201610,201611,201712,201711,201710
0,10754147.0,0.023342,0.023342,0.023342,0.023342,0.023342,0.023342
1,10759547.0,0.019459,0.019459,0.019459,0.019459,0.019459,0.019459
2,10843547.0,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000
3,10859147.0,0.035622,0.035622,0.035622,0.035622,0.035622,0.035622
4,10879947.0,0.020465,0.020465,0.020465,0.020465,0.020465,0.020465


In [25]:
e.ParcelId = e.ParcelId.astype(int)

In [26]:
e.to_csv('Only_CatBoost5678.csv', float_format='%.6f',index=False)